No Rio Grande do Norte, sabe-se que por muito tempo saúde e segurança foram problemas nunca tratados antes pelo poder público. Desde sempre tivemos números alarmantes relativos à saúde precária ou à falta de segurança. [Reportagens de jornais](https://oglobo.globo.com/brasil/rio-grande-do-norte-registra-maior-crescimento-da-taxa-de-homicidios-22749111) noticiam o crescimento da taxa de homícidios pelo estado com o passar dos anos. O maior hospital de traumas, queimados e urgentes do estado (Hospital Walfredo Gurgel) [segue lotado](http://blog.tribunadonorte.com.br/heitorgregorio/fila-de-ambulancias-por-macas-presas-no-hospital-walfredo-gurgel/). 

Desde 1996, o IBGE calcula a taxa de mortalidade no estado do RN e disponibiliza através de datasets no site DATASUS do Ministério da Saúde. Esta [taxa de mortalidade](http://tabnet.datasus.gov.br/tabdata/LivroIDB/2edrev/a10.pdf) contabiliza o número total de óbitos, por mil habitantes, na população residente em determinado espaço geográfico. 

Sendo assim, nesta linha de pensamento, buscou-se verificar por amostragem se a taxa de mortalidade nos municípios do estado do Rio Grande do Norte diminuiu, aumentou ou manteve-se estagnada ao longo dos anos.

No site [DATASUS](http://tabnet.datasus.gov.br/cgi/tabcgi.exe?sim/cnv/obt10RN.def) do governo federal, temos as taxas de mortalidade dos municípios do RN disponibilizadas no formato CSV, por município e código. O período da amostragem dos dados varia entre 1996 e 2017. A princípio iremos utilizar a taxa de mais antiga, de 1996, composta pelos **municípios** e pelos **Óbitos por Ocorrência**.

O site disponibiliza os dados puros em formato CSV, sem uso de API para aquisição via web, por essa forma carregaremos a planilha CSV manualmente no programa.

In [247]:
import pandas as pd

mort1996 = pd.read_csv('rn_mort_1996.csv', sep=';')
mort1996.head()

,Município,Óbitos p/Ocorrênc
0,240010 Acari,63
1,240020 Açu,215
2,240030 Afonso Bezerra,18
3,240040 Água Nova,1
4,240050 Alexandria,56


Vemos que os dados da columa município precisam ser tratados, existe um código associado ao munícipio e o próprio nome. Separaremos com o auxílio do seguinte trecho de código:

In [248]:
new = mort1996["Município"].str.split(" ", n = 1, expand = True) 
  
mort1996["ID"]= new[0] 
mort1996["Name"]= new[1] 
  
# Dropping old Name columns 
mort1996.drop(columns =["Município"], inplace = True) 

mort1996.head()  

,Óbitos p/Ocorrênc,ID,Name
0,63,240010,Acari
1,215,240020,Açu
2,18,240030,Afonso Bezerra
3,1,240040,Água Nova
4,56,240050,Alexandria


A fim de garantir a consistência dos dados, traremos o nome de todos os muncípios do RN através da API do IBGE:

In [0]:
# install the latest version of requests
!pip install requests==v2.21.0

In [249]:
import requests
import json # Import the JSON library.

# configure a generical header
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

#configure generical parameters
estado = 'RN'
estado_codigo = 24
estado_nome = 'Rio Grande do Norte'
estado_localizacao = [-5.3967435,-38.4230197]

response = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados", headers=headers)
json_data = response.json()

aux = {}
for jd in json_data:
  if(jd['nome'] == estado_nome):
    
    aux = jd.copy()
    break
print(aux)

{'id': 24, 'sigla': 'RN', 'nome': 'Rio Grande do Norte', 'regiao': {'id': 2, 'sigla': 'NE', 'nome': 'Nordeste'}}


In [250]:
response2 = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/estados/'+str(aux['id'])+'/municipios',headers=headers)
cities_json = response2.json()

cities_id = [cities['id'] for cities in cities_json]
cities_nome = [cities['nome'] for cities in cities_json]

df = pd.DataFrame(list(zip(cities_id,cities_nome)), columns =['ID', 'Name']) 
df.head()

,ID,Name
0,2400109,Acari
1,2400208,Açu
2,2400307,Afonso Bezerra
3,2400406,Água Nova
4,2400505,Alexandria


Agora que temos os dados advindos do IBGE, notamos que possuímos o ID e o nome de cada município, porém o ID associado à eles é diferente do anterior. O ID advindo da tabela do DATASUS possui 6 dígitos, o ID vindo deta tabela do IBGE possui 7 dígitos. Será que há diferença nos nomes dos municípios também? Anotamos apenas os nomes dos munícipios e através de ferramentas externas, podemos notar pelo [diagrama de venn](http://bioinfogp.cnb.csic.es/tools/venny/) que há diferenças nos nomes dos munícipios, existem 2 nomes de municípios que devem ser tratados (**Município ignorado - RN e
None**) nos dados do DATASUS e 17 munícipios anotados IBGE que não haviam sido reportados no ano de 1996. 
Pelos dados previamente analisados, foi decidido unir ambas as tabelas através de um DataFrame.merge() do Pandas nos moldes: left join e utilizando "Name" como coluna de ligação (existem 150 municípios com o nome igual), excluindo-se também a coluna com ID de 6 dígitos, pelo fato de estar fora do padrão do IBGE.

In [252]:
data = df.merge(mort1996, left_on='Name', right_on='Name')
data.drop(columns=['ID_y'], inplace = True)
data.rename(columns = {"ID_x": "ID", "Name":"Nome", "Óbitos p/Ocorrênc":"DADOS"}, inplace = True)

data_state = pd.DataFrame.from_dict(data)
data_state.head()

,ID,Nome,DADOS
0,2400109,Acari,63
1,2400208,Açu,215
2,2400307,Afonso Bezerra,18
3,2400406,Água Nova,1
4,2400505,Alexandria,56


Dados que temos a tabela, com dados desnecessários já tratados (pela natureza da união dos dataframes), prepararemos um colormap para contribuir na visualização do mapa. 

In [253]:
from branca.colormap import linear

# colormap yellow and green (YlGn)
minimo = min(data_state['DADOS'])
maximo = max(data_state['DADOS'])

colormap = linear.RdBu_04.scale(minimo, maximo)

colormap.caption="#Dados dos municípios do "+estado

colormap

Vamos adquirir as malhas dos municípios do RN para desenhar o mapa do RN utilizando as cores do colormap anterior para indiciar os municípios com as maiores taxas de mortalidade.

In [0]:
response = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/"+str(estado_codigo)+"/"+
                        "?formato=application/vnd.geo+json&resolucao=5",
                       headers=headers)
data_json = response.json()

In [255]:
import folium

# Create a map object
m_state = folium.Map(
    location=estado_localizacao,
    zoom_start=7,
    tiles='Stamen Terrain'
)

def function_x(x):
  color = None

  try:
    dados_aux = data_state.loc[data_state['ID']==int(x['properties']['codarea']), 'DADOS']
    dados = float(dados_aux.values[0])
    color = colormap(dados)
  except:
    dados = 0
    color = colormap(dados)
    pass
  color = colormap(dados)
  return {'fillColor': color, 'color': 'black','weight':1, 'fillOpacity':0.5}

# Create a Choropleth using folium.GeoJson()
folium.GeoJson(data_json,
               name='Dados',
               style_function=lambda x: function_x(x)
              ).add_to(m_state)

# Add a LayerControl.
folium.LayerControl().add_to(m_state)

# And the Color Map legend.
colormap.add_to(m_state)

m_state

Verificamos que o o colormap não foi suficiente para ser distinguido suficientemente bem entre os municípios do interior, devido à Natal e Mossoró serem pontos extremamente fora da curva. Para analisar melhor este dado entre os municípios do interior, removeremos Natal e Mossoró do nosso Dataset.

In [256]:
#Removendo o ponto fora da curva
to_drop = data_state.loc[data_state['Nome']=='Natal'].index.values

data_state.drop(df.index[to_drop], inplace=True)
to_drop = data_state.loc[data_state['Nome']=='Mossoró'].index.values

data_state.drop(df.index[to_drop], inplace=True)
data_state.head()

,ID,Nome,DADOS
0,2400109,Acari,63
1,2400208,Açu,215
2,2400307,Afonso Bezerra,18
3,2400406,Água Nova,1
4,2400505,Alexandria,56


Recriando o colormap

In [257]:
from branca.colormap import linear

# colormap yellow and green (YlGn)
minimo = min(data_state['DADOS'])
maximo = max(data_state['DADOS'])

colormap = linear.RdBu_04.scale(minimo, maximo)

colormap.caption="#Dados dos municípios do "+estado

colormap


Recriando o mapa

In [258]:
import folium

# Create a map object
m_state = folium.Map(
    location=estado_localizacao,
    zoom_start=7,
    tiles='Stamen Terrain'
)

def function_x(x):
  color = None
  aux = None
  try:
    dados_aux = data_state.loc[data_state['ID']==int(x['properties']['codarea']), 'DADOS']
    dados = float(dados_aux.values[0])
    color = colormap(dados)
  except:
    dados = 0
    color = colormap(dados)
    pass
  color = colormap(dados)
  return {'fillColor': color, 'color': 'black','weight':1, 'fillOpacity':0.5}

# Create a Choropleth using folium.GeoJson()
folium.GeoJson(data_json,
               name='Dados',
               style_function=lambda x: function_x(x)
              ).add_to(m_state)

# Add a LayerControl.
folium.LayerControl().add_to(m_state)

# And the Color Map legend.
colormap.add_to(m_state)

m_state

Agora iremos re-executar o código acima para os dados de 2017.

In [259]:
import pandas as pd

mort = pd.read_csv('rn_mort_2017.csv', sep=';')

new = mort["Município"].str.split(" ", n = 1, expand = True) 
  
mort["ID"]= new[0] 
mort["Name"]= new[1] 
  
# Dropping old Name columns 
mort.drop(columns =["Município"], inplace = True) 

import requests
import json # Import the JSON library.

# configure a generical header
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
}

#configure generical parameters
estado = 'RN'
estado_codigo = 24
estado_nome = 'Rio Grande do Norte'
estado_localizacao = [-5.3967435,-38.4230197]

response = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados", headers=headers)
json_data = response.json()

aux = {}
for jd in json_data:
  if(jd['nome'] == estado_nome):
    
    aux = jd.copy()
    break

response2 = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/estados/'+str(aux['id'])+'/municipios',headers=headers)
cities_json = response2.json()

cities_id = [cities['id'] for cities in cities_json]
cities_nome = [cities['nome'] for cities in cities_json]

df = pd.DataFrame(list(zip(cities_id,cities_nome)), columns =['ID', 'Name']) 

data = df.merge(mort, left_on='Name', right_on='Name')
data.drop(columns=['ID_y'], inplace = True)
data.rename(columns = {"ID_x": "ID", "Name":"Nome", "Óbitos p/Ocorrênc":"DADOS"}, inplace = True)
data_rn = pd.DataFrame.from_dict(data)

#Removendo o ponto fora da curva
to_drop = data_rn.loc[data_rn['Nome']=='Natal'].index.values
data_rn.drop(df.index[to_drop], inplace=True)
to_drop = data_rn.loc[data_rn['Nome']=='Mossoró'].index.values
data_rn.drop(df.index[to_drop], inplace=True)

from branca.colormap import linear

# colormap yellow and green (YlGn)
minimo = min(data_rn['DADOS'])
maximo = max(data_rn['DADOS'])

colormap = linear.RdBu_04.scale(minimo, maximo)

colormap.caption="#Dados dos municípios do "+estado

colormap

import folium

# Create a map object
m = folium.Map(
    location=estado_localizacao,
    zoom_start=7,
    tiles='Stamen Terrain'
)

def function_x(x):
  color = None
  try:
    dados_aux = data_rn.loc[data_rn['ID']==int(x['properties']['codarea']), 'DADOS']
    dados = float(dados_aux.values[0])
    color = colormap(dados)
  except:
    dados = 0
    color = colormap(dados)
    pass
  color = colormap(dados)
  return {'fillColor': color, 'color': 'black','weight':1, 'fillOpacity':0.5}

# Create a Choropleth using folium.GeoJson()
folium.GeoJson(data_json,
               name='Dados',
               style_function=lambda x: function_x(x)
              ).add_to(m)

# Add a LayerControl.
folium.LayerControl().add_to(m)

# And the Color Map legend.
colormap.add_to(m)

m

Analisando previamente os mapas que desenhamos de 1996 e 2017, considerando a remoção de Natal e Mossoró da análise, verificamos que os municípios que tiverem um aumento na taxa de mortalidade são justamente os municípios que hoje são considerados "grandes", ou pelo menos, maiores que a média. São eles: Parnamirim, Macaíba, São Gonçalo do Amarante, Ceará-Mirim, Caicó, Currais Novos e Pau dos Ferros. Aparentemente, o município de Açu teve uma diminuição na taxa de mortalidade quando considera-se apenas os períodos de 1996 e 2017. Faremos uma análise em relação à isso com o seguinte gráfico.

In [260]:
dt = data_rn

dt['DIFF'] = data_rn['DADOS']-data_state['DADOS']
  
dt.head()

,ID,Nome,DADOS,DIFF
0,2400109,Acari,50,-13.0
1,2400208,Açu,190,-25.0
2,2400307,Afonso Bezerra,40,22.0
3,2400406,Água Nova,13,12.0
4,2400505,Alexandria,61,5.0


In [261]:
from branca.colormap import linear

# colormap yellow and green (YlGn)
minimo = min(dt['DIFF'])
maximo = max(dt['DIFF'])

colormap = linear.RdBu_04.scale(minimo, maximo)

colormap.caption="#Dados dos municípios do "+estado

colormap

In [262]:
import folium

# Create a map object
m_dt = folium.Map(
    location=estado_localizacao,
    zoom_start=7,
    tiles='Stamen Terrain'
)

def function_x(x):
  color = None
  try:
    dados_aux = dt.loc[dt['ID']==int(x['properties']['codarea']), 'DIFF']
    dados = float(dados_aux.values[0])
    color = colormap(dados)
  except:
    dados = 0
    color = colormap(dados)
    pass
  return {'fillColor': color, 'color': 'black','weight':1, 'fillOpacity':0.5}

# Create a Choropleth using folium.GeoJson()
folium.GeoJson(data_json,
               name='Dados',
               style_function=lambda x: function_x(x)
              ).add_to(m_dt)

# Add a LayerControl.
folium.LayerControl().add_to(m_dt)

# And the Color Map legend.
colormap.add_to(m_dt)

m_dt

Analisando o mapa acima, chegamos a conclusão de que com o passar dos anos, a taxa de mortalidade aumentou nas maiores cidades do interior do estado, porém manteve-se "estagnada" para a maioria dos municípios do estado. 